In [17]:
# Packages

from gurobipy import Model, GRB, quicksum
import numpy as np

HAP set Generation

In [18]:
from itertools import product
n_teams = 10
n_minus_1 = n_teams - 1
opt_1 = 5
opt_2 = 4

def valid_pattern(pattern):
    # Count occurrences of 'H' and 'A'
    h_count = pattern.count('H')
    a_count = pattern.count('A')

    # Check for similar number of H and A matches (4H-5A or 5H-4A)
    if not (h_count == opt_1 and a_count == opt_2 or h_count == opt_2 and a_count == opt_1):
        return False

    # Check for more than two consecutive H or A matches
    for i in range(len(pattern) - 2):
        if pattern[i] == pattern[i + 1] == pattern[i + 2]:
            return False

    return True

# Generate all combinations of H and A for N-1 matches
all_patterns = product('HA', repeat=n_minus_1)

# Filter patterns that meet the criteria
valid_patterns = [''.join(pattern) for pattern in all_patterns if valid_pattern(pattern)]

# Output the number of valid patterns
print(f"Total valid patterns: {len(valid_patterns)}")


Total valid patterns: 90


 Defining the set of complimentary patterns

In [19]:
# Function to check if two patterns are complementary
def are_complementary(pattern1, pattern2):
    return all((c1 == 'H' and c2 == 'A') or (c1 == 'A' and c2 == 'H') for c1, c2 in zip(pattern1, pattern2))

# Find all pairs of complementary patterns
C = []
for i in range(len(valid_patterns)):
    for j in range(i + 1, len(valid_patterns)):
        if are_complementary(valid_patterns[i], valid_patterns[j]):
            C.append((valid_patterns[i], valid_patterns[j]))

print(C)

C_indices = [(valid_patterns.index(x), valid_patterns.index(y)) for x, y in C]

print(C_indices)


[('HHAHHAHAA', 'AAHAAHAHH'), ('HHAHHAAHA', 'AAHAAHHAH'), ('HHAHAHHAA', 'AAHAHAAHH'), ('HHAHAHAHA', 'AAHAHAHAH'), ('HHAHAHAAH', 'AAHAHAHHA'), ('HHAHAAHHA', 'AAHAHHAAH'), ('HHAHAAHAH', 'AAHAHHAHA'), ('HHAHAAHAA', 'AAHAHHAHH'), ('HHAAHHAHA', 'AAHHAAHAH'), ('HHAAHHAAH', 'AAHHAAHHA'), ('HHAAHAHHA', 'AAHHAHAAH'), ('HHAAHAHAH', 'AAHHAHAHA'), ('HHAAHAHAA', 'AAHHAHAHH'), ('HHAAHAAHH', 'AAHHAHHAA'), ('HHAAHAAHA', 'AAHHAHHAH'), ('HAHHAHHAA', 'AHAAHAAHH'), ('HAHHAHAHA', 'AHAAHAHAH'), ('HAHHAHAAH', 'AHAAHAHHA'), ('HAHHAAHHA', 'AHAAHHAAH'), ('HAHHAAHAH', 'AHAAHHAHA'), ('HAHHAAHAA', 'AHAAHHAHH'), ('HAHAHHAHA', 'AHAHAAHAH'), ('HAHAHHAAH', 'AHAHAAHHA'), ('HAHAHAHHA', 'AHAHAHAAH'), ('HAHAHAHAH', 'AHAHAHAHA'), ('HAHAHAHAA', 'AHAHAHAHH'), ('HAHAHAAHH', 'AHAHAHHAA'), ('HAHAHAAHA', 'AHAHAHHAH'), ('HAHAAHHAH', 'AHAHHAAHA'), ('HAHAAHHAA', 'AHAHHAAHH'), ('HAHAAHAHH', 'AHAHHAHAA'), ('HAHAAHAHA', 'AHAHHAHAH'), ('HAHAAHAAH', 'AHAHHAHHA'), ('HAAHHAHHA', 'AHHAAHAAH'), ('HAAHHAHAH', 'AHHAAHAHA'), ('HAAHHAHAA', 'AHHA

Finding feasible pattern opponents for each week

In [20]:
# Dictionary `feasible_opponents` where feasible_opponents[i, t]
# is the set of feasible opponents for pattern i in week t


# Initialize the feasible opponents dictionary
feasible_opponents = {}

# Iterate through each pattern and week
num_patterns = len(valid_patterns)
num_weeks = len(valid_patterns[0])

for i in range(num_patterns):
    for t in range(num_weeks):
        # Get the H/A status of pattern i in week t
        current_status = valid_patterns[i][t]
        
        # Find all patterns that have the opposite status in week t
        feasible_opponents[i, t] = {j for j in range(num_patterns) if j != i and valid_patterns[j][t] != current_status}


# Print the feasible opponents dictionary for verification
for key, value in feasible_opponents.items():
    print(f"Pattern {key[0]} in week {key[1]} can play against patterns {value}")


Pattern 0 in week 0 can play against patterns {45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89}
Pattern 0 in week 1 can play against patterns {15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89}
Pattern 0 in week 2 can play against patterns {15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89}
Pattern 0 in week 3 can play against patterns {8, 9, 10, 11, 12, 13, 14, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 69, 70, 71, 72, 73, 74, 82, 83, 84, 85, 86, 87, 88, 89}
Pattern 0 in week 4 can play against patterns {2, 3, 4, 5, 6, 7, 15, 16, 17, 18, 19, 20, 28, 2

In [ ]:

from gurobipy import Model, GRB, quicksum

# Initialize model
m = Model("Sports_Scheduling")

# Define Parameters
P = range(len(valid_patterns))  # Pattern indices: p, p'
W = range(n_minus_1)           # Week indices: w
N = n_teams

# Binary decision variable: r[p, p', w] = 1 if pattern p is paired with pattern p' in week w
r_var = m.addVars(P, P, W, vtype=GRB.BINARY, name="r")

# Binary decision variable: y[p] = 1 if pattern p is selected
y_var = m.addVars(P, vtype=GRB.BINARY, name="y")

# 1) Select exactly N patterns:  sum_{p in P} y_p = N
m.addConstr(quicksum(y_var[p] for p in P) == N, name="select_N_patterns")

# 2) Complementary patterns must both be selected:  y_p - y_p' = 0  for all (p, p') in C_indices
for (p, pp) in C_indices:
    m.addConstr(y_var[p] - y_var[pp] == 0, name=f"complementary_{p}_{pp}")

# 3) For each pattern p and week w, it must be paired with exactly one opponent if y[p] = 1:
#    sum_{p' in Pi_{p,w}} r_{p,p',w} = y_p
#    Here, feasible_opponents[p, w] corresponds to Π_{p,w} in our model.
for p in P:
    for w in W:
        # Ensure the sum of r_var over all valid opponents p' in week w equals y[p].
        # We split it into r_var[p, pp, w] and r_var[pp, p, w] depending on p<pp or p>pp.
        m.addConstr(
            quicksum(r_var[p, pp, w] for pp in feasible_opponents[p, w] if p < pp) +
            quicksum(r_var[pp, p, w] for pp in feasible_opponents[p, w] if pp < p)
            == y_var[p],
            name=f"pattern_consistency_{p}_{w}"
        )


# 4) Each pair (p, pp) can be used in at most one week if p < pp:
#    sum_{w in W} r_{p,p',w} <= y_p
for p in P:
    for pp in P:
        if p < pp:
            m.addConstr(
                quicksum(r_var[p, pp, w] for w in W) <= y_var[p],
                name=f"unique_match_{p}_{pp}"
            )

# Restrict r_var[p, pp, w] to 0 if p >= pp to avoid duplications (p' < p in our model)
for p in P:
    for pp in P:
        if p >= pp:
            for w in W:
                r_var[p, pp, w].ub = 0

# Similarly, set r_var[p, pp, w] = 0 if pp is not a feasible opponent for p in week w
for p in P:
    for w in W:
        for pp in P:
            if p < pp:
                if pp not in feasible_opponents[p, w]:
                    r_var[p, pp, w].ub = 0

In [22]:
# Solve the model
m.setParam("MIPFocus", 1)
m.optimize()

# Retrieve the solution if optimal
if m.status == GRB.OPTIMAL:
    # r_sol captures all r_{p,p',w} = 1
    r_sol = {
        (p, pp, w): r_var[p, pp, w].x
        for p in P for pp in P for w in W
        if r_var[p, pp, w].x > 0.5
    }
    # y_sol captures the selected patterns
    y_sol = {
        p: y_var[p].x
        for p in P
        if y_var[p].x > 0.5
    }

    solution_found = {
        "r": r_sol,
        "y": y_sol
    }
    print("Optimal solution found.")
else:
    print("No optimal solution found.")

Set parameter MIPFocus to value 1
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 4861 rows, 72990 columns and 77490 nonzeros
Model fingerprint: 0xd522aeba
Variable types: 0 continuous, 72990 integer (72990 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]
Presolve removed 171 rows and 54720 columns
Presolve time: 0.41s
Presolved: 4690 rows, 18270 columns, 59283 nonzeros
Variable types: 0 continuous, 18270 integer (18270 binary)

Root relaxation: objective 0.000000e+00, 1970 iterations, 0.56 seconds (0.37 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It

In [23]:
import pandas as pd

# Extract weeks dynamically
weeks = sorted({t for _, _, t in r_sol.keys()})
calendar_data = {f"Week {t+1}": [] for t in weeks}

# Populate calendar data for each week 
for t in weeks:
    week_pairs = []
    for (i, j, w) in r_sol:
        if w == t:
            week_pairs.append(f"Pattern {i} vs Pattern {j}")
    calendar_data[f"Week {t+1}"] = week_pairs

# Create and display the DataFrame
calendar_df = pd.DataFrame(calendar_data)
calendar_df

,Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week 7,Week 8,Week 9
0,Pattern 35 vs Pattern 49,Pattern 35 vs Pattern 53,Pattern 35 vs Pattern 50,Pattern 35 vs Pattern 54,Pattern 35 vs Pattern 52,Pattern 35 vs Pattern 39,Pattern 35 vs Pattern 36,Pattern 35 vs Pattern 37,Pattern 35 vs Pattern 40
1,Pattern 36 vs Pattern 52,Pattern 36 vs Pattern 50,Pattern 36 vs Pattern 49,Pattern 36 vs Pattern 53,Pattern 36 vs Pattern 40,Pattern 36 vs Pattern 54,Pattern 37 vs Pattern 53,Pattern 36 vs Pattern 39,Pattern 36 vs Pattern 37
2,Pattern 37 vs Pattern 54,Pattern 37 vs Pattern 49,Pattern 37 vs Pattern 52,Pattern 37 vs Pattern 50,Pattern 37 vs Pattern 39,Pattern 37 vs Pattern 40,Pattern 39 vs Pattern 40,Pattern 40 vs Pattern 49,Pattern 39 vs Pattern 50
3,Pattern 39 vs Pattern 53,Pattern 39 vs Pattern 52,Pattern 39 vs Pattern 54,Pattern 39 vs Pattern 49,Pattern 49 vs Pattern 53,Pattern 49 vs Pattern 52,Pattern 49 vs Pattern 50,Pattern 50 vs Pattern 52,Pattern 49 vs Pattern 54
4,Pattern 40 vs Pattern 50,Pattern 40 vs Pattern 54,Pattern 40 vs Pattern 53,Pattern 40 vs Pattern 52,Pattern 50 vs Pattern 54,Pattern 50 vs Pattern 53,Pattern 52 vs Pattern 54,Pattern 53 vs Pattern 54,Pattern 52 vs Pattern 53


In [24]:
solution_found

{'r': {(35, 36, 6): 1.0,
  (35, 37, 7): 1.0,
  (35, 39, 5): 1.0,
  (35, 40, 8): 1.0,
  (35, 49, 0): 1.0,
  (35, 50, 2): 1.0,
  (35, 52, 4): 1.0,
  (35, 53, 1): 1.0,
  (35, 54, 3): 1.0,
  (36, 37, 8): 1.0,
  (36, 39, 7): 1.0,
  (36, 40, 4): 1.0,
  (36, 49, 2): 1.0,
  (36, 50, 1): 1.0,
  (36, 52, 0): 1.0,
  (36, 53, 3): 1.0,
  (36, 54, 5): 1.0,
  (37, 39, 4): 1.0,
  (37, 40, 5): 1.0,
  (37, 49, 1): 1.0,
  (37, 50, 3): 1.0,
  (37, 52, 2): 1.0,
  (37, 53, 6): 1.0,
  (37, 54, 0): 1.0,
  (39, 40, 6): 1.0,
  (39, 49, 3): 1.0,
  (39, 50, 8): 1.0,
  (39, 52, 1): 1.0,
  (39, 53, 0): 1.0,
  (39, 54, 2): 1.0,
  (40, 49, 7): 1.0,
  (40, 50, 0): 1.0,
  (40, 52, 3): 1.0,
  (40, 53, 2): 1.0,
  (40, 54, 1): 1.0,
  (49, 50, 6): 1.0,
  (49, 52, 5): 1.0,
  (49, 53, 4): 1.0,
  (49, 54, 8): 1.0,
  (50, 52, 7): 1.0,
  (50, 53, 5): 1.0,
  (50, 54, 4): 1.0,
  (52, 53, 8): 1.0,
  (52, 54, 6): 1.0,
  (53, 54, 7): 1.0},
 'y': {35: 1.0,
  36: 1.0,
  37: 1.0,
  39: 1.0,
  40: 1.0,
  49: 1.0,
  50: 1.0,
  52: 1.0,
 